In [6]:
import pandas as pd

## Fast Finalization using SQL



### Create the Database

Run the following code to create an SQLite3 file called 'gapminder.db', which contains the gapminder data.

This is the file we'll be pulling data from!

In [91]:
import bokeh
# bokeh.sampledata.download()
from bokeh.sampledata import gapminder
for name in ['population', 'fertility', 'life_expectancy', 'regions']:
    df = getattr(gapminder, name)
    if 'Group' in df:
        df = df.rename(columns={'Group': 'Continent'})
    df.to_sql(name, 'sqlite:///gapminder.db', if_exists='replace')


## Queries in SQL using Pandas

Reading in data from an sql database in Pandas is done using the SQLAlchemy library.  Instead of a filename, you'll need to give a [SQLAlchemy connection string](https://docs.sqlalchemy.org/en/14/core/engines.html#database-urls) to the read function:

| Function | Purpose | Example |
| :----    | :-----  | :------ |
| `pd.read_sql_table(table_name, conn_string)` | Reads a whole table in a database | `pd.read_sql_table('people', 'sqlite:///census.db'` |
| `pd.read_sql_query(sql, conn_string)` | Send an sql command to a database | `pd.read_sql_query('SELECT * FROM people', 'sqlite:///census.db'` |


### Exploring a Database File

Every type of SQL Datbase has a slightly different "dialect" of SQL (e.g. SQLite3, PostGreSQL, MySQL), but most SQL is similar.
The biggest difference in the dialect comes up when asking meta questions about the database, two of which are very important 
when just getting started:

   1. What Databases (e.g. "Schemas") can I get from this connection?
   2. What Tables are in this Schema?
   3. What Columns are in a table?
   
Each of this questions can be asked via a SQL query, and the answer is always a table of its own!

SQLite3 only can contain a single database, so we'll look at the last two questions:

| Question | Function | 
| :----    | :-----   |
| What Tables are there? | `SELECT * FROM sqlite_master` |
| What Columns are in this table? | `PRAGMA table_info('MyTable')`

**Exercises**

Use the `pd.read_sql_query()` function to answer the following questions about the organization of the "gapminder.db" SQLite3 database.

What Tables are in the database?

In [189]:
pd.read_sql_query("SELECT * FROM sqlite_master", "sqlite:///gapminder.db")

,type,name,tbl_name,rootpage,sql
0,table,population,population,2,"CREATE TABLE population (\n\t""Country"" TEXT, \..."
1,index,ix_population_Country,population,3,"CREATE INDEX ""ix_population_Country"" ON popula..."
2,table,fertility,fertility,20,"CREATE TABLE fertility (\n\t""Country"" TEXT, \n..."
3,index,ix_fertility_Country,fertility,21,"CREATE INDEX ""ix_fertility_Country"" ON fertili..."
4,table,life_expectancy,life_expectancy,50,"CREATE TABLE life_expectancy (\n\t""Country"" TE..."
5,index,ix_life_expectancy_Country,life_expectancy,51,"CREATE INDEX ""ix_life_expectancy_Country"" ON l..."
6,table,regions,regions,80,"CREATE TABLE regions (\n\t""Country"" TEXT, \n\t..."
7,index,ix_regions_Country,regions,81,"CREATE INDEX ""ix_regions_Country"" ON regions (..."


What columns are in the population table?

In [192]:
pd.read_sql_query("PRAGMA table_info('population')", "sqlite:///gapminder.db").head()

,cid,name,type,notnull,dflt_value,pk
0,0,Country,TEXT,0,None,0
1,1,1964,FLOAT,0,None,0
2,2,1965,FLOAT,0,None,0
3,3,1966,FLOAT,0,None,0
4,4,1967,FLOAT,0,None,0


What columns are in the fertility table?

In [193]:
pd.read_sql_query("PRAGMA table_info('fertility')", "sqlite:///gapminder.db").head()

,cid,name,type,notnull,dflt_value,pk
0,0,Country,TEXT,0,None,0
1,1,1964,FLOAT,0,None,0
2,2,1965,FLOAT,0,None,0
3,3,1966,FLOAT,0,None,0
4,4,1967,FLOAT,0,None,0


What columns are in the regions table?

In [194]:
pd.read_sql_query("PRAGMA table_info('regions')", "sqlite:///gapminder.db").head()

,cid,name,type,notnull,dflt_value,pk
0,0,Country,TEXT,0,None,0
1,1,Continent,TEXT,0,None,0
2,2,ID,TEXT,0,None,0


### Reading a Full Table: `pd.read_sql_table(table_name, conn)`

If you know what table you want to read in, all you need is the table name and you're off!



**Exercises**

Use `pd.read_sql_table` to read each table from the database:

The fertility table:

In [195]:
pd.read_sql_table('fertility', 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.395,5.141,4.900
1,Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.748,1.760,1.771
2,Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.829,2.820,2.795
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The population table

In [196]:
pd.read_sql_table('population', 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,10474903.0,10697983.0,10927724.0,11163656.0,11411022.0,11676990.0,11964906.0,12273101.0,12593688.0,...,26693486.0,27614718.0,28420974.0,29145841.0,29839994.0,30577756.0,31411743.0,32358260.0,33397058.0,34499915.0
1,Albania,1817098.0,1869942.0,1922993.0,1976140.0,2029314.0,2082474.0,2135599.0,2188650.0,2241623.0,...,3124861.0,3141800.0,3156607.0,3169665.0,3181397.0,3192723.0,3204284.0,3215988.0,3227373.0,3238316.0
2,Algeria,11654905.0,11923002.0,12229853.0,12572629.0,12945462.0,13338918.0,13746185.0,14165889.0,14600659.0,...,32396048.0,32888449.0,33391954.0,33906605.0,34428028.0,34950168.0,35468208.0,35980193.0,36485828.0,36983924.0
3,American Samoa,22672.0,23480.0,24283.0,25087.0,25869.0,26608.0,27288.0,27907.0,28470.0,...,61871.0,62962.0,64045.0,65130.0,66217.0,67312.0,68420.0,69543.0,70680.0,71834.0
4,Andorra,17438.0,18529.0,19640.0,20772.0,21931.0,23127.0,24364.0,25656.0,26997.0,...,75292.0,77888.0,79874.0,81390.0,82577.0,83677.0,84864.0,86165.0,87518.0,88909.0


The regions table

In [197]:
pd.read_sql_table('regions', 'sqlite:///gapminder.db').head()

,Country,Continent,ID
0,Angola,Sub-Saharan Africa,AO
1,Benin,Sub-Saharan Africa,BJ
2,Botswana,Sub-Saharan Africa,BW
3,Burkina Faso,Sub-Saharan Africa,BF
4,Burundi,Sub-Saharan Africa,BI


The life expectancy table

In [198]:
pd.read_sql_table('life_expectancy', 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,33.639,34.152,34.662,35.170,35.674,36.172,36.663,37.143,37.614,...,56.583,57.071,57.582,58.102,58.618,59.124,59.612,60.079,60.524,60.947
1,Albania,65.475,65.863,66.122,66.316,66.500,66.702,66.948,67.251,67.595,...,75.725,75.949,76.124,76.278,76.433,76.598,76.780,76.979,77.185,77.392
2,Algeria,47.953,48.389,48.806,49.205,49.592,49.976,50.366,50.767,51.195,...,69.682,69.854,70.020,70.180,70.332,70.477,70.615,70.747,70.874,71.000
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Querying a Database: `pd.read_sql_query()`

If you have a more-detailed request you want to make, you can send an SQL query to the database
and get only the data you really want to work with.  Most queries in sql start with **SELECT**;
What kinds of queries can you make?  A lot! Here's a cheat sheet for basic queries:

[<img src="https://ugoproto.github.io/ugodoc/img/sql_nosql/SQL-Cheatsheet-2.png" width=620></img>](https://ugoproto.github.io/ugodoc/img/sql_nosql/SQL-Cheatsheet-2.png)


**Tip:**  These queries can get quite long; using python docstrings can help make it easer to read. for example:

```python
query = """
SELECT Name, Age FROM People
WHERE Age > 5
LIMIT 10
"""
pd.read_sql_query(query, 'sqlite:///data.db')
```

**Exercises**

Write SQL Queries that get only the data asked for below:

Get all the fertility data.

In [199]:
query = """
SELECT * FROM fertility
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.395,5.141,4.900
1,Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.748,1.760,1.771
2,Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.829,2.820,2.795
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get all the popualtion data.

Get all the regions data

Print the 1980 fertility data.

In [28]:
query = """
SELECT "1980", "1981" FROM fertility
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


Print the 2010 life expectancy data

Print the 1990 and 2010 and population data

Print Italy's Population data.

In [31]:
query = """
SELECT * FROM fertility
WHERE Country = "Italy"
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Italy,2.5,2.517,2.524,2.524,2.515,2.497,2.469,2.429,2.377,...,1.29,1.315,1.341,1.367,1.392,1.415,1.435,1.454,1.471,1.487


Print Germany's Population data.

Print Mexico and Cuba's 1990 fertility data

In [228]:
query = """
SELECT Country, "1990" FROM fertility
WHERE Country = "Mexico" OR Country = "Cuba"
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country,1990
0,Cuba,1.752
1,Mexico,3.375


In [229]:
query = """
SELECT Country, "1990" FROM fertility
WHERE Country in ("Mexico", "Cuba")
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country,1990
0,Cuba,1.752
1,Mexico,3.375


Print the first 3 rows of the life expectancy table

In [210]:
query = """
SELECT * FROM life_expectancy
LIMIT 5
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,33.639,34.152,34.662,35.170,35.674,36.172,36.663,37.143,37.614,...,56.583,57.071,57.582,58.102,58.618,59.124,59.612,60.079,60.524,60.947
1,Albania,65.475,65.863,66.122,66.316,66.500,66.702,66.948,67.251,67.595,...,75.725,75.949,76.124,76.278,76.433,76.598,76.780,76.979,77.185,77.392
2,Algeria,47.953,48.389,48.806,49.205,49.592,49.976,50.366,50.767,51.195,...,69.682,69.854,70.020,70.180,70.332,70.477,70.615,70.747,70.874,71.000
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Print the first 3 rows of 1970's population data

Print the first 7 rows of 1990's fertility data

Show the Countries that had fertility rates above 6.5 people in 2004

In [242]:
query = """
SELECT Country, "2004" FROM fertility
WHERE "2004" > 6.5
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

Show the countries that had fertility rates below 1.4 in 2010.

Show the countries that had populations above 100 million in 1980. What were their populations in 2006?

Show the countries that are listed as being in "Europe & Central Asia"

### Joining Data

What if you want to compare and combine data from multiple tables?  The "Join" command is used the same way as Pandas' Merge function: comparing two columns and matching on them. 

Get the fertility and population rates for all countries in 1980:

In [223]:
query = """
SELECT 
  p.Country, 
  p."1989" pop1989, 
  f."1989" fert1989 
FROM population p
INNER JOIN fertility f ON p.Country = f.Country
ORDER BY f."2010"
LIMIT 5
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Country,pop1989,fert1989
0,American Samoa,45595.0,None
1,Andorra,51241.0,None
2,Anguilla,7964.0,None
3,Bermuda,59416.0,None
4,British Virgin Islands,15926.0,None


In [160]:
query = """
SELECT fertility.Country, fertility."1980" fert1980, population."1980" pop1980 FROM fertility
INNER JOIN population ON fertility.Country = population.Country
LIMIT 4
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Country,fert1980,pop1980
0,Afghanistan,7.669,14185729.0
1,Albania,3.678,2671300.0
2,Algeria,6.889,18811199.0
3,American Samoa,NaN,32526.0


In [172]:
# aliasing name of table to reduce typing
query = """
SELECT f.Country, f."1980" fert1980, p."1980" pop1980 FROM fertility f
INNER JOIN population p ON f.Country = p.Country
LIMIT 4
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Country,fert1980,pop1980
0,Afghanistan,7.669,14185729.0
1,Albania,3.678,2671300.0
2,Algeria,6.889,18811199.0
3,American Samoa,NaN,32526.0


Get the life expectancy and fertility rates for all countries in 1990:

Get the life expectancy and fertility rates for all France in 1988:

Get the life expectancy in 2005 for all countries with fertility rates in 2005 less than 1.6.
Which countries are included?

Get the population of all countries with life expectancies of greater than 80 years in 2000.

Get the fertility rates of all countries in Europe

Get the population of all countries in Sub-saharan Africa

### Group By: Doing Statistics through SQL

https://www.sqlitetutorial.net/sqlite-group-by/

Aggregation functions: `AVG()`, `SUM()`, `MIN()`, `MAX()`

In [182]:
query = """
SELECT regions.Continent, regions.Country, AVG(population."1980") meanPop1980 FROM population
INNER JOIN regions ON population.Country = regions.Country
GROUP BY Continent
ORDER BY meanPop1980
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Continent,Country,meanPop1980
0,Sub-Saharan Africa,Angola,7.636512e+06
1,Middle East & North Africa,Algeria,8.896094e+06
2,America,Anguilla,1.233459e+07
3,Europe & Central Asia,Albania,1.254172e+07
4,East Asia & Pacific,American Samoa,3.629319e+07
5,South Asia,Afghanistan,1.132596e+08


Which continents had the highest average population in 1990?

Which continents had the 2 lowest fertility rates in 2000?